In [ ]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [ ]:
import torch

repo = "isl-org/ZoeDepth"
model_zoe_nk = torch.hub.load(repo, "ZoeD_NK", pretrained=False)
pretrained_dict = torch.hub.load_state_dict_from_url('https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt', map_location='cpu')
model_zoe_nk.load_state_dict(pretrained_dict['model'], strict=False)
for b in model_zoe_nk.core.core.pretrained.model.blocks:
    b.drop_path = torch.nn.Identity()

Using cache found in /root/.cache/torch/hub/isl-org_ZoeDepth_main


img_size [384, 512]


Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


Params passed to Resize transform:
	width:  512
	height:  384
	resize_target:  True
	keep_aspect_ratio:  True
	ensure_multiple_of:  32
	resize_method:  minimal


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
Downloading: "https://github.com/isl-org/ZoeDepth/releases/download/v1.0/ZoeD_M12_NK.pt" to /root/.cache/torch/hub/checkpoints/ZoeD_M12_NK.pt
100%|██████████| 1.35G/1.35G [00:04<00:00, 344MB/s]


In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import colormaps
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize
import os
import numpy as np
from zoedepth.utils.misc import pil_to_batched_tensor, save_raw_16bit, colorize

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model_zoe_nk.to(DEVICE)

folder_path = "/content/drive/MyDrive/FaultLines/svi/40187_size300x800_fov120/"
output_path = "/content/depthmap_svi/"

for filename in os.listdir(folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path).convert("RGB")
        X = pil_to_batched_tensor(image).to(DEVICE)

        depth_numpy = model_zoe_nk.infer_pil(image)  # Assuming it returns a numpy array

        norm = Normalize(vmin=depth_numpy.min(), vmax=depth_numpy.max())
        colormap = plt.cm.rainbow  # Can change colormap here
        colored_depth = colormap(norm(depth_numpy))


        depth_image = Image.fromarray((colored_depth[:, :, :3] * 255).astype(np.uint8))


        save_path = os.path.join(output_path, filename)
        depth_image.save(save_path)

        print(f"Processed and saved: {save_path}")

Processed and saved: /content/depthmap_svi/SVI-2-IxOrYAVftoqWWvuS0dkzNw-40.99266-28.8421-40187RES3711-40187COM3722.jpg
Processed and saved: /content/depthmap_svi/SVI-3-IxOrYAVftoqWWvuS0dkzNw-40.99267-28.84216-40187RES3736-40187RES3711.jpg
Processed and saved: /content/depthmap_svi/SVI-1-IMOVbdE-V5qqsaWZCMJC5Q-40.9924-28.84312-40187COM3678-40187RES3677.jpg
Processed and saved: /content/depthmap_svi/SVI-5-HpWPrMpEyVjyqUm5TFkfWQ-40.9927-28.84284-40187RES3743-40187RES3688.jpg
Processed and saved: /content/depthmap_svi/SVI-4-IxOrYAVftoqWWvuS0dkzNw-40.99276-28.84212-40187RES3727-40187RES3740.jpg
Processed and saved: /content/depthmap_svi/SVI-13-JEu5TNzjQdFl7sjVRs_J_w-40.99321-28.84166-40187RES3796-40187RES3785.jpg
Processed and saved: /content/depthmap_svi/SVI-14-gRodA_r-7QeYhRJd19ShSQ-40.99362-28.84155-40187RES3767-40187RES3816.jpg
Processed and saved: /content/depthmap_svi/SVI-9-KAE37vrdKzuh_xb8zUwEJA-40.99312-28.84181-40187RES3716-40187RES3716.jpg
Processed and saved: /content/depthmap_sv

In [ ]:
import zipfile



In [73]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

zip_file_path = '/content/svi_calc.zip'
target_dir = '/content/drive/MyDrive/FaultLines/svi/40187_size300x800_fov120/calc'

# Create a zip file
with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir(target_dir, zipf)